[Node 18: Synchronisation](http://www-static.etp.physik.uni-muenchen.de/kurs/Computing/python2/node18.html)

Navigation:

 **Next:** [Python Threads und GIL](node19.ipynb) **Up:** [Threads and Multi–Processing](node16.ipynb) **Previous:** [Die Thread Klasse](node17.ipynb)

Wir betrachten ein weitere Anwendungsbeispiel für threading -- in diesem sehr vereinfachten Beispiel zählen zwei Threads einen Counter parallel hoch.

In [ ]:
import threading 
  
class Counter(object):
    def __init__(self):
        self.num = 0
    def inc(self):
        self.num += 1

def thread_task(counter):
    for _ in range(100000):
        counter.inc()
  
# global counter
counter = Counter()

# create two threads 
t1 = threading.Thread(target = thread_task, args = (counter,))
t2 = threading.Thread(target = thread_task, args = (counter,))
  
# start threads 
t1.start() 
t2.start() 
  
# wait until threads finish their job 
t1.join() 
t2.join() 
  
# print result
print(counter.num) 


Probieren Sie es aus und zeigen Sie auf, wenn Sie das Ergebnis haben.